In [1]:
import pandas as pd 
df = pd.read_csv('imdb.zip')

In [2]:
df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [3]:
import joblib
tk = joblib.load('tokenizer.pkl')

In [5]:
seqs = tk.texts_to_sequences(df['review'])

In [7]:
seqs[0]

[4, 27, 27, 27, 287, 407, 1217, 13, 36, 4, 1218, 1219, 408, 142]

In [8]:
import tensorflow as tf

In [9]:
# padding 
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs)

In [11]:
pads[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    4,   27,   27,   27,  287,  407, 1217,
         13,   36,    4, 1218, 1219,  408,  142])

In [12]:
# 학습된 단어 임베딩 불러오기 
import numpy as np 
z = np.load('word-emb.npz')
# emb로 가중치를 저장했기 때문에 이런식으로 불러와야함 
e = z['emb']

In [13]:
e

array([[-0.00585933, -0.02039977, -0.01549274, ..., -0.00398161,
         0.0175071 , -0.01756992],
       [ 0.32903343, -0.3360473 , -0.30365658, ...,  0.35520142,
         0.35840875,  0.2751575 ],
       [ 0.23425817, -0.31939816, -0.27791482, ...,  0.28666493,
         0.31019026,  0.29353744],
       ...,
       [-0.0060727 , -0.01226221, -0.07054661, ...,  0.01590607,
         0.063531  ,  0.06172537],
       [ 0.06765597, -0.02405636, -0.05426505, ...,  0.01865944,
         0.07510275,  0.05638415],
       [-0.01524026,  0.01175383,  0.04869963, ..., -0.04706771,
         0.02162607,  0.04247859]], dtype=float32)

In [14]:
df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [16]:
# 감성분석 모델 만들기 

# 단어 모형의 임베딩된 가중치들을 시작값으로 넣어주기 (전이 학습의 핵심) 
emb2 = tf.keras.layers.Embedding(input_dim=tk.num_words + 1, output_dim=8,
                             embeddings_initializer=tf.keras.initializers.Constant(e)) # <--- 이렇게 써줘야함 

model = tf.keras.Sequential([
    emb2,
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 8)           16008     
_________________________________________________________________
global_average_pooling1d (Gl (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 8)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 16,089
Trainable params: 16,089
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(optimizer='adam',
             metrics=['acc'],
             loss='binary_crossentropy')

In [19]:
y = df['sentiment'].values

In [20]:
model.fit(pads, y)

32/32 [==============================] - 0s 904us/step - loss: 0.6927 - acc: 0.5000
